## Criando uma classe iterável, para tokenização e geração de sentenças 
    - Recupera tokens em português, que normalmente tokenizer de frameworks, falham em capturar, como por exemplo palavras compostas como dias da semana: terça-feira, segunda-feira; e outras palavras separadas por traço "-".
    - Código com otimização de memória, utilizando geradores ao invés de armazenar valores na memória, utlizando funções geradoras (sintáxe muito similar a de list comprehension)
        
        - Sintáxe função geradora -> (x for x in 'some iterable here like a list, tuple, ...')
        
        - Exemplo de função geradora  def exemplo(): return (x for x in [1,2,3,4,5])
        
            - Exemplo utilização -> list( exemplo() ) -> Retorna a lista, sendo gerada de forma "lazy".

## Requisitos
    
    - pip install spacy 
    - python -m spacy download pt_core_news_sm

In [1]:
import re
import spacy
from string import punctuation


stop_words_portuguese = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no',
 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito',
 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos',
 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa',
 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês',
 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas',
 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos',
 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos',
 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos', 'estiverem',
 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 'houveram', 'houvera', 'houvéramos', 'haja', 'hajamos', 'hajam',
 'houvesse', 'houvéssemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houverá', 'houveremos',
 'houverão', 'houveria', 'houveríamos', 'houveriam', 'sou', 'somos', 'são', 'era', 'éramos', 'eram', 'fui', 'foi',
 'fomos', 'foram', 'fora', 'fôramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fôssemos', 'fossem', 'for', 'formos',
 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 'seríamos', 'seriam', 'tenho', 'tem', 'temos', 'tém', 'tinha',
 'tínhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham',
 'tivesse', 'tivéssemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'terá', 'teremos', 'terão', 'teria',
 'teríamos', 'teriam']


nlp = spacy.load('pt_core_news_sm')

PATTERN = r"""\w+-+\w+|\w+|[!'"#$%&\*\+,\./:;\?]{1}"""


/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:

class TextNLP:
    
    """
    Classe iterável, para recuperação de tokens e sentenças.
    Exemplo de uso:
        
        # Inicializando a variável
        >> w = TextNLP('Hoje é terça-feira, e o lobo é mau! E nós também somos! Somos? Será que somos...')
        
        # Visualizando o objeto criado
        >> w
          Texto: Hoje é terça-feira, e o lobo é mau! E nós também somos! Somos? Será que somos...
          Tokens: ['Hoje', 'é', 'terça-feira', ',', 'e', 'o', 'lobo', 'é', 'mau', '!', 'E', 'nós', 'também', 'somos', '!', 'Somos', '?', 'Será', 'que', 'somos', '.', '.', '.']
          Num Tokens: 23
          
        # Recuperando apenas os tokens
        >> w.words
          ['Hoje', 'é', 'terça-feira', ',', 'e', 'o', 'lobo', 'é', 'mau', '!', 'E', 'nós', 'também', 'somos', '!', 'Somos', '?', 'Será', 'que', 'somos', '.', '.', '.']
    
        # Preprocessando os tokens, retirando stop words, pontuação, ou ambos (Default)
        >> list(w.words_preprocessing())
          ['Hoje', 'é', 'terça-feira', 'lobo', 'é', 'mau', 'E', 'Somos', 'Será']

        # Apenas stop words
        >> list(w.words_preprocessing(punctuation_remove=False))
          ['Hoje', 'é', 'terça-feira', ',', 'lobo', 'é', 'mau', '!', 'E', '!', 'Somos', '?', 'Será', '.', '.', '.']
          
        # Apenas pontuação
        >> list(w.words_preprocessing(stop_words_remove=False))
          ['Hoje', 'é', 'terça-feira', 'e', 'o', 'lobo', 'é', 'mau', 'E', 'nós', 'também', 'somos', 'Somos', 'Será', 'que', 'somos']

        # Recuperação de sentenças
        >> for sent in w.sents:
           .... print(sent, ' <end sentence>')
           
           
           "
               Hoje é terça-feira, e o lobo é mau!  <end sentence>
               E nós também somos!  <end sentence>
               Somos?  <end sentence>
               Será que somos...  <end sentence>
           "   

    """
    def __init__(self, text):
        assert isinstance(text, str), "'A classe deve ser inicializada com uma string sendo passada por parâmetro'"
        
        if text == '':
            raise ValueError('A string passada não pode estar vazia')
        
        self.text = text
        
        
    def __iter__(self):
        
        return (match.group() for match in re.finditer(PATTERN, self.text))
    
    
    def __repr__(self):
        tokens = self.words
        
        return "Texto: %s\nTokens: %s\nNum Tokens: %d" % (self.text, tokens, len(tokens))
    
    
    @property
    def words(self):
            
        return list(self.__iter__())
    
    
    def words_preprocessing(self, stop_words_remove=True, punctuation_remove=True):
        
        tokens = list(self.__iter__())
        
        if stop_words_remove:
            tokens = (tok for tok in tokens if tok not in stop_words_portuguese)
        
        if punctuation_remove:
            tokens = (tok for tok in tokens if tok not in punctuation)
            
        return tokens
    
    
    @property
    def sents(self):
        doc = nlp(self.text)
                
        return (sent for sent in doc.sents)

In [3]:
w = TextNLP('Hoje é terça-feira, e o lobo é mau! E nós também somos! Somos? Será que somos...')
list(w.words_preprocessing(stop_words_remove=False))

['Hoje',
 'é',
 'terça-feira',
 'e',
 'o',
 'lobo',
 'é',
 'mau',
 'E',
 'nós',
 'também',
 'somos',
 'Somos',
 'Será',
 'que',
 'somos']

In [4]:
list(w.words_preprocessing(punctuation_remove=False))

['Hoje',
 'é',
 'terça-feira',
 ',',
 'lobo',
 'é',
 'mau',
 '!',
 'E',
 '!',
 'Somos',
 '?',
 'Será',
 '.',
 '.',
 '.']

In [5]:
# Default
list(w.words_preprocessing())

['Hoje', 'é', 'terça-feira', 'lobo', 'é', 'mau', 'E', 'Somos', 'Será']

In [6]:
w

Texto: Hoje é terça-feira, e o lobo é mau! E nós também somos! Somos? Será que somos...
Tokens: ['Hoje', 'é', 'terça-feira', ',', 'e', 'o', 'lobo', 'é', 'mau', '!', 'E', 'nós', 'também', 'somos', '!', 'Somos', '?', 'Será', 'que', 'somos', '.', '.', '.']
Num Tokens: 23

In [7]:
w.words

['Hoje',
 'é',
 'terça-feira',
 ',',
 'e',
 'o',
 'lobo',
 'é',
 'mau',
 '!',
 'E',
 'nós',
 'também',
 'somos',
 '!',
 'Somos',
 '?',
 'Será',
 'que',
 'somos',
 '.',
 '.',
 '.']

In [8]:
for item in w.words:
    print(item)

Hoje
é
terça-feira
,
e
o
lobo
é
mau
!
E
nós
também
somos
!
Somos
?
Será
que
somos
.
.
.


In [9]:
for sent in w.sents:
    print(sent, ' <end sentence>')

Hoje é terça-feira, e o lobo é mau!  <end sentence>
E nós também somos!  <end sentence>
Somos?  <end sentence>
Será que somos...  <end sentence>
